# Load Dataset

In [1]:
from datasets import load_dataset

ds = load_dataset("rajpurkar/squad")

In [2]:
# Structure
print(ds)
print("\nTraining set:", ds["train"][0])
print("\nTesting set:", ds["validation"][0])

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

Training set: {'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues an

In [26]:
ds["validation"]["answers"]["text"]

Column([['Denver Broncos', 'Denver Broncos', 'Denver Broncos'], ['Carolina Panthers', 'Carolina Panthers', 'Carolina Panthers'], ['Santa Clara, California', "Levi's Stadium", "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."], ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'], ['gold', 'gold', 'gold']])

In [ ]:
answer_list = ds["validation"]["answers"]["text"]
print(answer_list)

# [{}...{}]
tmp_answer_list1 = [set(sublist1) for sublist1 in answer_list]

# [[]...[]]
tmp_answer_list2 = [list(subset) for subset in tmp_answer_list1]

# []
tmp_answer_list3 = ["; ".join(sublist2) for sublist2 in tmp_answer_list2]
tmp_answer_list3

# Tokenization

In [41]:
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorWithPadding

# Tokenization
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")


def tokenization(example):
    questions = example["question"]
    
    answer_list = example["answers"]
    tmp_answer_list1 = [subdict['text'] for subdict in answer_list]  # [[]...[]]
    answers = ["; ".join(sublist) for sublist in tmp_answer_list1]  # []
    
    return tokenizer(questions, answers=answers, truncation=True)


train_tokenized_ds = ds['train'].map(tokenization, batched=True)
valid_tokenized_ds = ds['validation'].map(tokenization, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

<class 'list'>
<class 'list'>


TypeError: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'answers'

In [ ]:
# Tokenization
if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))

train_tokenized_ds = ds["train"].map(lambda e: tokenizer(e["prompt"], truncation=True,
                                                         padding='max_length'), batched=True)
valid_tokenized_ds = ds["validation"].map(lambda e: tokenizer(e["prompt"], truncation=True,
                                                              padding='max_length'),
                                          batched=True)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")